In [1]:
import xlwings as xw
import pandas as pd
wb_cy = xw.Book('cytiva.xlsm')
SHEET_NAMES =  ['Temp_DB', 'Shipment information', '인수증', 
'대리점송장', '대리점 출고대기', '로컬리스트', 'In-Transit part report', '기타리스트',
 '출고리스트', 'Cytiva Inventory BIN']
import win32com.client as cli

outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
inbox = outlook.GetDefaultFolder(6) # 받은편지함
msg = inbox.Items #메세지 정보
msg_counter = msg.count # 총갯수

In [8]:
shape_name = 'SVC_CCH_23011901'
idx = int(shape_name.split('_')[2])
idx

23011901

In [36]:
ws_main = wb_cy.sheets['통합제어']
idx_col = ws_main.range('J12:J500').options(numbers=int).value
ws_main.range('J12').column

10

In [13]:
row_nm = idx_col.count(idx)

In [34]:
chs_rng = ws_main.range(11 +row_nm,17)
current_status = chs_rng.value
current_status

'REQUESTED'

In [27]:
ML_STATUS= ['REQUESTED','PROCESSING', 'SHIPPED']

ML_FOLDERS = ['inbox','1_Requests', '2_Processing', '3_ShipConfirmed']

In [39]:
ml_dict =dict(zip(ML_STATUS,ML_FOLDERS[1:]))
ml_dict[current_status]

'1_Requests'

In [49]:
argu_folder = inbox.Parent.Folders(ml_dict[current_status])

In [50]:
msg = argu_folder.Items
part_request = []
service_request = 'SVC'
for ms in msg:
    if service_request in ms.Subject:
        part_request.append(ms)
# 서비스만 추출하기



In [52]:
part_request[0].Subject

'SVC_CCH_23011901'

In [56]:
def get_mail_status(shape_name=None):
    """
    메일제목을 argument로 받으면 현재상태의 해당 메일의 현재 ML_BIN, STATUS를 반환 
    """
    ws_main= wb_cy.sheets['통합제어']
    
    ml_status = ['REQUESTED','PROCESSING', 'SHIPPED']
    ml_folders = ['inbox','1_Requests', '2_Processing', '3_ShipConfirmed']
    
    ml_dict = dict(zip(ml_status,ml_folders[1:]))
    
    if shape_name != None:
        
        shape_name = shape_name.replace('_prfm','')

        idx = int(shape_name.split('_')[2])
        emp_row_nm = get_empty_row(col='J') ## J = 10, ML_INDEX 컬럼
        idx_list = ws_main.range("J12:J"+str(emp_row_nm+1)).options(numbers=int).value
        row_nm = idx_list.count(idx)
        status_cel = ws_main.range(11 + row_nm,17).value ## Q=17, STATUS 컬럼 , 11 ML_INDEX의 row 번호
        fd_bin = ml_dict[status_cel]
        ms_list = Email.get_email_obj(fd_bin)
    
    return ms_list
        

In [54]:
def get_email_obj(folder_name = 'inbox'):

    outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩

    if folder_name == 'inbox':

        argu_folder = outlook.GetDefaultFolder(6) # 받은편지함
    else:
        argu_folder = outlook.GetDefaultFolder(6).Parent.Folders(folder_name) # 1번폴더


    msg = argu_folder.Items #메세지 정보
    part_request = []
    service_request = 'SVC'
    for ms in msg:
        if service_request in ms.Subject:
            part_request.append(ms)
    # 서비스만 추출하기

    return part_request

In [55]:
def get_empty_row(sheet=wb_cy.selection.sheet,col=1):
    """
    특정컬럼의 마지막 값의 행번호 구하기
    """
    sel_sht = sheet
    col_num = col
    if type(col) == int :
        row_start_nm = sel_sht.range(1048576,col_num).end('up').row + 1 
    elif type(col) == str :
        row_start_nm = sel_sht.range(col+str(1048576)).end('up').row + 1 
    return row_start_nm

In [ ]:
get_mail_status()

In [11]:
# 제목입력 받으면 oracle도 조회
import cx_Oracle
import os

In [13]:
LOCATION = r"C:\Users\lms46\Desktop\fulfill\instantclient_fulfill"
os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
cx_Oracle.init_oracle_client(lib_dir=LOCATION)

connection = cx_Oracle.connect(
    user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

)
cursor = connection.cursor()

In [61]:
ml_sub = 'SVC_CCH_23011901'

In [90]:
ml_status_db = cursor.execute(f'select * from MAIL_STATUS where ML_SUB = (:name1)',name1=ml_sub)

In [91]:
for ml in ml_status_db:
    df_status = pd.DataFrame(ml)

In [104]:
df_status.T[[1,2,3,4]]

,1,2,3,4
0,SVC_CCH_23011901,REQUESTED,2023-01-23 21:50:30,1_Requests


In [ ]:
argu = 'SVC_CCH_23011901'

In [7]:
def DataWarehouse():
    try:
        os.environ["PATH"] = LOCATION + ";" + os.environ["PATH"]
        cx_Oracle.init_oracle_client(lib_dir=LOCATION)
        connection = cx_Oracle.connect(
            user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

        )
        cursor = connection.cursor()
        return cursor

    except:

        connection = cx_Oracle.connect(
            user='dw_fulfill', password='fulfillment123QWE!@#', dsn='fulfill_high'

        )
        cursor = connection.cursor()
        return cursor

In [8]:
def get_mail_status(ml_sub):
    """
    메일제목을 argument로 받으면 현재상태의 해당 메일의 현재 ML_BIN, STATUS를 반환 
    """
    ml_sub = ml_sub.replace('_prfm','')
    cur = DataWarehouse()
    query = 'select * from MAIL_STATUS where ML_SUB = (:name1)'
    db_obj = cur.execute(query, name1= ml_sub)
    df_status = pd.DataFrame(db_obj.fetchall())
    # 최신 날짜로 업데이트된 부분만 가져오기
    df_status = df_status.sort_values(3,ascending=False).iloc[[0]][[1,2,3,4]]
    return df_status


In [179]:
get_mail_status(ml_sub)

,1,2,3,4
0,SVC_CCH_23011901,REQUESTED,2023-01-23 21:50:30,1_Requests


In [178]:
msg = get_email_obj(get_mail_status(ml_sub)[4][0])

find_ms = []
for ms in msg :
    if ms.Subject == ml_sub:
        find_ms.append(ms)
find_ms[0].Subject

'SVC_CCH_23011901'

In [133]:
df_status = get_mail_status()
df_status

""


In [130]:
df_status[3] = pd.to_datetime(df_status[3], format='%Y-%m-%d %H:%M:%S', errors='raise')

In [131]:
df_status[3]

0   2023-01-23 21:50:30
Name: 3, dtype: datetime64[ns]

In [117]:
tmp_rows = cursor.execute('select * from MAIL_STATUS')
rows_tmp = []
for row in tmp_rows:
    rows_tmp.append(row)
pd.DataFrame(rows_tmp)

,0,1,2,3,4
0,7,SVC_CCH_23011901,REQUESTED,2023-01-23 21:50:30,1_Requests
1,8,SVC_LMR_23011902,REQUESTED,2023-01-23 21:50:30,1_Requests
2,9,SVC_LWW_23011903,REQUESTED,2023-01-23 21:50:30,1_Requests


In [174]:
tmp_rows = cursor.execute('select * from MAIL_STATUS')
df_status = pd.DataFrame(tmp_rows.fetchall())
df_status[3] = pd.to_datetime(df_status[3], format='%Y-%m-%d %H:%M:%S', errors='raise')

In [175]:
df_status = df_status.sort_values(3,ascending=False).iloc[[0]]

In [176]:
df_status

,0,1,2,3,4
0,7,SVC_CCH_23011901,REQUESTED,2023-01-23 21:50:30,1_Requests


# 프린팅 

- 받는 agru는 ml_sub

1. 프린트하기 누르면 프린트 3장
    - **프린트가 되지 않은상태에서는 아래 모든 작업들이 이루어져서는 안된다.**
    - 다시프린트 하는 경우에는 아래 프로세스는 진행하면 안됨
2. 프린트 완료되면, 해당 상태 PROCESSING으로 업데이트 및 메일도 2_Processing으로 이동 move_mail(), insert_data()
3. excel상에도 PROCESSING으로 변경 
4. SHIP_CONFIRM 컬럼에 버튼 생성
5. excel 상에서 SHIPPED 및 MAIL_STATUS SHIPPED 변경 및 해당 메일 3_ShipConfirmed 폴더로 이동  move_mail(), insert_data()

In [2]:
from datetime import datetime

In [3]:
def move_mail(from_fd,to_fd,req_type="SVC",ml_index=None):
    """
    메일 폴더간 이동 모듈///
    """
    outlook = cli.Dispatch("Outlook.Application").GetNamespace("MAPI") # 아웃룩
    inbox = outlook.GetDefaultFolder(6) # 받은편지함
    request_folder = outlook.GetDefaultFolder(6).Parent.Folders('1_Requests') # 1번폴더
    process_folder = outlook.GetDefaultFolder(6).Parent.Folders('2_Processing') # 2번폴더
    shipped_folder = outlook.GetDefaultFolder(6).Parent.Folders('3_ShipConfirmed') # 3번폴더

    
    ml_folders = ['inbox','1_Requests', '2_Processing', '3_ShipConfirmed']

    fd_dict = {

        ml_folders[0]:inbox, ml_folders[1]:request_folder, ml_folders[2]:process_folder,ml_folders[3]:shipped_folder
    }


    from_fd = fd_dict[from_fd]
    to_fd = fd_dict[to_fd]

    part_request = []
    if ml_index == None:
        for ms in from_fd.Items:
            if req_type in ms.Subject:
                part_request.append(ms)
    else :
        for ms in from_fd.Items:
            if ml_index in ms.Subject:
                part_request.append(ms)
        
        # from_folder내용이 없을 경우
        if len(part_request) == 0:
            return None
        else :
            for ms in part_request:
                if req_type in ms.Subject:
                    ms.Move(to_fd)

In [108]:
def get_mail_status(ml_sub):
    """
    메일제목을 argument로 받으면 현재상태의 해당 메일의 현재 ML_BIN, STATUS를 반환 
    """
    ml_sub = ml_sub.replace('_prfm','')
    cur = DataWarehouse()
    query = 'select * from MAIL_STATUS where ML_SUB = (:name1)'
    db_obj = cur.execute(query, name1= ml_sub)
    df_status = pd.DataFrame(db_obj.fetchall())
    # 최신 날짜로 업데이트된 부분만 가져오기
    df_status = df_status.sort_values(3,ascending=False).iloc[[0]].reset_index(drop=True)
#     df_status = df_status.sort_values(3,ascending=False).iloc[[0]][[1,2,3,4]]
    return df_status

In [96]:
ml_index = 'SVC_LWW_23011903'
ml_status = ['REQUESTED','PROCESSING', 'SHIPPED']
ml_folders = ['inbox','1_Requests', '2_Processing', '3_ShipConfirmed']

ml_dict = dict(zip(ml_status,ml_folders[1:]))
ml_status_to = 'PROCESSING'
bin_folder_to = '2_Processing'

In [111]:
# def update_status(self,ml_index,ml_status,bin_folder,req_type="SVC"):
current_df = get_mail_status(ml_index)

In [112]:
current_df

,0,1,2,3,4
0,21,SVC_LWW_23011903,SHIPPED,2023-01-26 23:49:13,3_ShipConfirmed


In [15]:
str(datetime.now()).split('.')[0]

'2023-01-26 21:36:00'

In [16]:
# 해당 인덱스가 어떤 행에 있는지 확인해야함
def get_empty_row(sheet=wb_cy.selection.sheet,col=1):
    """
    특정컬럼의 마지막 값의 행번호 구하기
    """
    sel_sht = sheet
    col_num = col
    if type(col) == int :
        row_start_nm = sel_sht.range(1048576,col_num).end('up').row + 1 
    elif type(col) == str :
        row_start_nm = sel_sht.range(col+str(1048576)).end('up').row + 1 
    return row_start_nm

In [17]:
ws_main= wb_cy.sheets['통합제어']
last_row = get_empty_row(col='J')
last_row

15

In [18]:
idx_list = ws_main.range((12,"J"),(last_row-1,'J')).options(numbers=int).value
idx_nm = idx_list.index(ml_index)
idx_list[idx_nm]

'SVC_CCH_23011901'

In [19]:



# 엑셀상 STATUS "Q"
ws_main.range(12+idx_nm,17).value = ml_status[1]

In [22]:
# SHIP_CONFIRM "S" 버튼생성
sh_conf= ws_main.range(12+idx_nm,19)

cel_left_sf = sh_conf.left
cel_top_sf = sh_conf.top
cel_width_sf = sh_conf.width
cel_height_sf = sh_conf.height

ws_main.api.Shapes.AddShape(125, cel_left_sf, cel_top_sf, cel_width_sf, cel_height_sf)

ws_main.shapes[-1].name = ml_index+'_shcf'
# 생성된 ship_confrim 버튼 객체
obj_shcf = ws_main.shapes[ml_index+'_shcf']
obj_shcf.text = '발송완료'
obj_shcf.api.TextFrame.HorizontalAlignment = 2
obj_shcf.api.TextFrame.VerticalAlignment = 2

In [33]:
# 메일 이동
move_mail('1_Requests','2_Processing',ml_index=ml_index)

In [66]:
# 현재시간
now = str(datetime.now()).split('.')[0]
# 메일 상태 db 업데이트
# index 는 None이어야 자동 증분
current_df[0][0] = None
# 상태 업데이트
current_df[2][0] = ml_status_to
# 업데이트시간
current_df[3][0] = now
# 아웃룩상 폴더
current_df[4][0] = bin_folder_to
current_df

C:\Users\lms46\AppData\Local\Temp\ipykernel_26744\975290178.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_df[2][0] = ml_status_to
C:\Users\lms46\AppData\Local\Temp\ipykernel_26744\975290178.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_df[3][0] = now
C:\Users\lms46\AppData\Local\Temp\ipykernel_26744\975290178.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_df[4][0] = bin_folder_to


,0,1,2,3,4
0,NaN,SVC_CCH_23011901,PROCESSING,2023-01-26 22:53:14,2_Processing


In [81]:
current_df[2][0]

'PROCESSING'

In [67]:
list_c_df = list(current_df.loc[0])

In [70]:
list_c_df[0]=None

In [76]:
df_ms = pd.DataFrame(list_c_df).T
type(df_ms[3][0])

str

In [79]:
ws_main.range((12+idx_nm,'Q'))

<Range [cytiva.xlsm]통합제어!$Q$12>

# Ship_confirm하기
1. ship_confirm 매서드생성
    - 메일연결 기능생성시 메일 status에따라 생성된다.
    - print btn을 누르면 생성되는 버튼
    - index + "_shcf"으로 버튼이름은 생성된다.
2. 해당 기능은 준비한 품목이 배송팀으로 정상 전달되었거나, 직접 발송을 하였을 경우 진행된다.
    - 모든 db 이동 및 excel상의 상태변화는 실제 행하는 일이 일어난 경우에 진행된다.
3. 메일 이동 프로세싱 에서 쉽컨펌드로
4. STATUS는 쉰컨펌드로
5. 발송완료 버튼은 비활성화 
    - if status == 쉽컨펌드 -> 아무런 반응 없도록
6. MAIL_STAUS ship_confrimed로 변환 
7. GOGOX상의 시스템으로 해당 배송건을 넘겨받는다면 해당 배송건은 추적가능 할 수있다.
    - GOGOX의 API를 넘겨받는 조건하에 가능..
   

In [87]:
xl_state = ws_main.range((12+idx_nm,'Q')).options(numbers=int).value
xl_state
current_df[2][0]

'PROCESSING'